ModuleNotFoundError: No module named 'bson'

In [71]:
# Import the Twython class
import tweepy
import csv
import time
import json
import datetime
import pandas as pd
import emoji
import os
from json import dumps
import preprocessor as p

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [40]:
#Mysql
import pymysql
import json
from functools import partial
import csv

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro
connection = pymysql.connect(
    host='localhost',
    user='megadados',
    password='linux123',
    database='redes')

db = partial(run_db_query, connection)


In [41]:
#Função do DB para retornar as tags dos jogadores
def return_players_tag(connection,id_jogador):    

    with connection.cursor() as cursor:

        cursor.execute('SELECT tag FROM jogador_tag WHERE id_jogador = %s', (id_jogador) ) ## irá retornar a tag do jogador no twitter

        res = cursor.fetchall()

        tags = tuple(x[0] for x in res)

        return tags

In [42]:
db('SELECT * FROM jogador_tag')


Executando query:
(1, 'alaatweets')
(3, 'kaj33')
(6, 'ShareefAbdur_R')
(10, 'alexabrines')
(15, 'QuincyAcy')
(19, '10jadams')
(20, 'jordanadams1231')
(22, 'RealStevenAdams')
(24, 'Bam1of1')
(27, 'Adrien4')
(29, 'moeager')
(32, 'danielrainge')
(33, 'AjincaAlexis42')
(40, 'furkanaldemir19')
(41, 'colea45')
(42, 'aldridge_12')
(44, 'CAlexander')
(52, 'Iam_RawleAlkins')
(54, 'GraysonJAllen')
(55, '_bigjayy_')
(60, 'THEleekAllen30')
(62, 'greenRAYn20')
(63, 'aa000G9')
(67, 'FreeMoAlmond')
(71, 'JohnAmaechi')
(73, 'farouq1')
(74, 'LouAmundson17')
(78, 'ASqared06')
(88, 'JusAnderson1')
(89, 'chibbs_1')
(91, 'KyleAnderson5')
(103, 'IkePostrs_')
(106, 'Giannis_An34')
(107, 'Kostas_ante13')
(109, 'carmeloanthony')
(110, 'GregAnthony50')
(113, 'OAnunoby')
(118, 'RobertArchibald')
(119, 'RyArch15')
(121, 'agentzeroshow')
(122, 'TrevorAriza')
(125, 'bjarmstrong')
(136, 'DarrellArthur00')
(138, 'melartis1')
(139, 'AsikOmer')
(157, 'Gustavo_Ayon15')
(159, 'DeandreAyton')
(160, 'KAzubuike7')
(162, 'Re

In [64]:
tags = return_players_tag(connection,582)
print(tags[0])
user_tweet = tags[0]

user_tweet_trade = tags[0] +  "Trade"
print(user_tweet_trade)

JimmyButler
JimmyButlerTrade


In [78]:

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    with open("twitter_credentials.json", "r") as file:
        creds = json.load(file)

    
    
    def __init__(self, keys_dict=creds, api=api,
                 result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(
            creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
        auth.set_access_token(
            creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
    
        
        #Date that starts after NBA finals until the beggining of the season
        start_date = datetime.datetime(2019, 6, 30, 00, 00, 00)
        end_date = datetime.datetime(2019, 10, 22, 00, 00, 00)

        api = tweepy.API(auth)

        #for tweet in tweepy.Cursor(api.user_timeline, screen_name=user_tweet, since=start_date, until=end_date).items():
        #    print("ID TWEET: " + str(tweet.id))
        
        self.result_limit = result_limit
    
        
    # remove special elements and provide as .text_clean_
    # indicate whether elements are woven into text (just in a word order sense for now)
    def clean(self):
        clean_text = {}
        split_text = dict(enumerate(self.text_.split()))
        special_elements = self.hashtags_ + self.handles_ + self.urls_ + self.emojis_
        for index, term in split_text.items():
            if term not in special_elements:
                clean_text.update({index:term})
        self.text_clean_ = " ".join(clean_text.values())

        # also check if any non-text elements are interwoven with text
        # not ideal to iterate twice but start/end are initially unknown (w/o rewrite of attr_ functions)
        self.is_complex_ = False
        clean_text_start = min(clean_text.keys())
        clean_text_end = max(clean_text.keys())
        for index, term in split_text.items():
            if term in special_elements and clean_text_start > index < clean_text_end:
                self.is_complex_ = True 
                
        
                
    # make a list of hashtags
    def find_hashtags(self):
        hashtags = []
        try:
            hashtags.extend(re.findall(r"#\w+", self.text_))
        except:
            pass
        self.hashtags_ = hashtags

    # list of handles
    def find_handles(self):
        handles = []
        try:
            handles.extend(re.findall(r"@\w+", self.text_))
        except:
            pass
        self.handles_ = handles

    # list of urls
    def find_urls(self):
        urls = []
        # source: https://www.geeksforgeeks.org/python-check-url-string/
        urls = re.findall(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", self.text_)
        self.urls_ = urls

    # list of emojis
    def find_emojis(self):
        emojis = []
        try:
            emojis.extend(re.findall(emoji.get_emoji_regexp(), self.text_))
        except:
            pass
        self.emojis_ = emojis
        self.demojis_ = [emoji.demojize(e) for e in emojis]
        
    # simple status report
    def show_attributes(self):
        for k,v in sorted(vars(self).items()):
            print(f"{k}: {v}", end='\n')
        print()
                
            
    def mine_user_tweets(self, user=user_tweet_trade, #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                    mined['text'] = p.clean(item.full_text)
                    mined['text'] = re.sub(r"http\S+", "", mined[''])
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 
#def save_to_json(tweet):
    #with open("saved_tweets.json", "wt") as file:
        #json.dump(tweet, file, indent = 4)

In [79]:
miner=TweetMiner(result_limit = 200)

In [80]:
mined_tweets = miner.mine_user_tweets(user=user_tweet,max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [81]:

handle_list= ['list of handles you want the timelines of']

twitter_dict={}
counter=0

for name in handle_list:
    try:
      twitter_dict[name]=[]
      twitter_dict[name].append(miner.mine_user_tweets(user=name, max_pages=17))
      counter = counter +1
      if counter%40==0:
        time.sleep(900) #15 minute sleep time
    #if name invalid print name and remove key
    except:
      print(name, 'is invalid or locked')
      twitter_dict.pop(name)
    
all_tweets=pd.concat([pd.DataFrame(twitter_dict[i][0]) for i in twitter_dict])

In [82]:
pd.DataFrame(mined_tweets)


,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
0,1191213266619551744,Jimmy Butler,JimmyButler,2048,together we can do so much https://t.co/2E4yUq...,2019-11-05 17:54:43.105838,2019-11-04 04:39:09,12158,[],5361,None,Twitter for iPhone,None,None,None
1,1189404191766302720,Jimmy Butler,JimmyButler,1622,"heat fans thank you for the support tonight, w...",2019-11-05 17:54:43.105852,2019-10-30 04:50:32,14424,[],5361,None,Twitter for iPhone,None,None,None
2,1188955535615365121,Jimmy Butler,JimmyButler,243,bringin’ the heat with my dude @IAmJustise in ...,2019-11-05 17:54:43.105857,2019-10-28 23:07:44,1289,"[{'text': 'Zaddy', 'indices': [120, 126]}, {'t...",5361,None,Twitter for iPhone,None,None,None
3,1188529034428403712,Jimmy Butler,JimmyButler,5529,family is not an important thing. it’s everyth...,2019-11-05 17:54:43.105861,2019-10-27 18:52:58,26907,[],5361,None,Twitter for iPhone,None,None,None
4,1188248564226580480,Jimmy Butler,JimmyButler,2259,good w @MiamiHEAT,2019-11-05 17:54:43.105865,2019-10-27 00:18:28,14573,[],5361,Place(_api=<tweepy.api.API object at 0x1222070...,Twitter for iPhone,None,None,None
5,1176406463092330496,Jimmy Butler,JimmyButler,643,up and at em,2019-11-05 17:54:43.105869,2019-09-24 08:02:12,4374,[],5361,Place(_api=<tweepy.api.API object at 0x1222070...,Twitter for iPhone,None,None,None
6,1176287426089762816,Jimmy Butler,JimmyButler,851,FOUR ANTE MERIDIEM. https://t.co/RbZzP66CAt,2019-11-05 17:54:43.105873,2019-09-24 00:09:11,4531,[],5361,Place(_api=<tweepy.api.API object at 0x1222070...,Twitter for iPhone,None,None,None
7,1171580357281492992,Jimmy Butler,JimmyButler,67,My recent trip to Senegal https://...,2019-11-05 17:54:43.105878,2019-09-11 00:24:58,417,[],5361,None,Twitter for iPhone,None,None,None
8,1160690287825608704,Jimmy Butler,JimmyButler,520,realonepointblankperiod @paulpogba https://t.c...,2019-11-05 17:54:43.105882,2019-08-11 23:11:43,4356,[],5361,Place(_api=<tweepy.api.API object at 0x1222070...,Twitter for iPhone,None,None,None
9,1146275152474324992,Jimmy Butler,JimmyButler,519,Great win brasil,2019-11-05 17:54:43.105886,2019-07-03 04:31:07,6348,[],5361,None,Twitter for iPhone,None,None,None


In [77]:
# Salvar json tweets

try:
    os.remove("saved_tweets.json")
except:
    pass


mined_tweets_df.to_json(r'saved_tweets.json')


